# GBDT 梯度提升树 

GBDT，梯度提升树，（Gradient Boosting Decision Tree）是一种基于Boosting集成学习思想的**加法模型**，训练时采用前向分布算法进行贪婪的学习，每次迭代都学习一颗CART树来**拟合之前t-1课树的预测结果与训练样本真实值的残差**。

回顾一下以前的知识：

1. CART回归树模型：
$$f(x)=\sum_{m=1}^M c_m I(x\in R_m)$$

$M$：数据集划分成M个单元

$c_m$: 第M个单元的输出值

$I(x\in R_m)$: 指示函数


2. 集成学习

集成学习可以通过构建并结合多个学习器来完成学习任务。根据个体学习器的生成方式，可以分成两大类：

- Bagging: 个体学习器之间不存在强依赖关系，可同时生成的并行化方法。如：随机森林。
- Boosting： 个体学习器之间存在强依赖关系，必须串行生成的序列化防范。如：AdaBoost、GDBT、XGBoost。

3. Boosting工作机制如下：

- 先从初始训练集训练出一个基学习器；
- 再根据基学习器的表现对训练样本分布进行调整，**使得先前基学习器做错的训练样本在后续受到更多关注**；
- 然后基于调整后的样本分布来训练下一个基学习器；
- 如此重复进行，直至基学习器数目达到事先指定的值T；
- 最终将这T个基学习器进行加权结合。


4. Boosting的最大好处在于，每一步的残差计算其实变相的增大的分错的样本的权重。


## 1. 提升树

提升树：以决策树（分类树或回归树）为基函数的提升方法称为提升决策树（Boosting Decision Tree），简称提升树（Boosting Tree）。

### 提升树模型：

提升树模型可以表示为决策树的加法模型：
$$f_M(x)=\sum_{m=1}^M T_m(x)$$
其中，$T_m(x)$表示第m决策树，M为树的个数。

### 提升树算法（回归问题）

输入：训练数据集 $T=\{(x_1,y_1),(x_2,y_2),...,(x_n,y_n)\}, x_i \in X \subseteq R^n, y_i \in Y \subseteq R $;

输出：提升树 $f_M(x)$。

1. 初始化 $f_0(x)=0$
2. 对$m=1,2,...,M$
    - 计算残差（残差是真实值与预测值的差）：
    $$r_{mi}=y_i - f_{m-1}(x_i), i=1,2,...,N$$
    - 拟合残差$r_{mi}$学习一个回归树，得到$T_m(x)$
    - 更新 $f_m(x)=f_{m-1}(x)+T_m(x)$。（加法模型与前向分布算法）
3. 得到回归问题提升树
    $$f_M(x)=\sum_{m=1}^M T_m(x)$$


### 提升树的训练优化目标（特征选择依据：平方误差最小）：

$$\min_s [\min_{c_1} \sum_{x_j\in R_1}{(y_i-c_1)^2} + \min_{x_i\in R_2}{(y_i-c_2)^2}]$$

其中：

- s是数据的切分点：
$$R_1 = \{x|x\leq s\}, R_2 = \{x|x\gt s\}$$

- $c_1$是$R_1$内部使平方损失误差达到最小值：
$$c_1=\frac{1}{N_1}\sum_{x_i\in R_i}{y_i}$$

这里 $N_1$是$R_1$的样本点数。

## 2. GBDT 梯度提升决策树

**核心思想：每棵树都去拟合上颗树的负梯度。在GBDT，无论损失函数是什么形式，每个决策树拟合的都是负梯度。准确的说，不是用负梯度代替残差，而是当损失函数是均方误差时，负梯度刚好是残差，残差只是特例。**

**GBDT的求解过程就是梯度下降在函数空间中的优化过程**

输入：训练数据集$T=\{(x_1,y_1),(x_2,y_2),...,(x_ny_n)\}, x_i\in X \subseteq R^n, y_i \in \subseteq R$; 损失函数 $L(y,f(x))$

输出：回归树 $\hat{f}(x)$

1. 初始化若学习器

$$f_0(x) = arg \min_c{\sum_{i=1}^N L(y_i, c)}$$

假设取损失函数为平方损失，因为平方损失函数是一个凸函数，直接对$c$求导：
$$\sum_{i=1}^N{\frac{\partial L(y_i, c)}{\partial c}} = \sum_{i=1}^N\frac{\partial (\frac{1}{2}(y_i-c)^2)}{\partial c} = \sum_{i=1}^N{(c-y_i)}$$

令导数等于0，得：

$$c = \frac{\sum_{i=1}^N{y_i}}{N}$$

所以初始化时，c取值为所有训练样本标签值的均值。此时得到初始化学习器：
$$f_0(x)=c$$

2. 迭代训练 m=1,2,...,M颗树：

- 对每个样本$i=1,2,...,N$，计算负梯度，即残差：
$$r_{mi}=-[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}]_{f(x)=f_{m-1}(x)}$$

- 将上步得到的残差 $r_{mi}$ 作为样本新的真实值，并将数据$(x_i, r_{mi}), i=1,2,...,N$作为下棵树的训练数据集，得到一颗新的回归树，其对应的叶子节点区域 $R_{mj}, j=1,2,...,J$，其中$J$为回归树的叶子节点的个数。

- 对$j=1,2,...,J$个叶子节点，计算最佳拟合值 ($c_{mj}$是$R_{mj}$的平方损失最小值)：
$$c_{mj}=arg \min_c \sum_{x_i\in R_{mj}}{L(y_i, f_{m-1}(x_i)+c)}$$

- 更新强学习器：
$$f_m(x)=f_{m-1}(x)+\sum_{j=1}^J{c_{mj}I(x\in R_{mj})}$$

3. 得到最终学习器GBDT：
$$\hat{f}(x)=f_M(x)=f_0(x)+\sum_{m=1}^M{\sum_{j=1}^J{c_{mj}I(x\in R_{mj})}}$$

- GBDT只能由回归树组成。
- GBDT只能串行生成。
- GBDT对异常值比较敏感
- GBDT是减少模型的偏差
- GBDT需要进行特征归一化